In [34]:
import markdown
import sys


def get_git_root(path):

        git_repo = git.Repo(path, search_parent_directories=True)
        git_root = git_repo.git.rev_parse("--show-toplevel")
        return (git_root)
    
    
git_def_path = get_git_root(os.getcwd())

git_path = git_def_path

documents_path = 'warehouse_docs'

filename = 'field_descriptions.md'

droughty_project = os.path.join(git_path,documents_path,filename)

with open(droughty_project) as f:
    htmlmarkdown=markdown.markdown( f.read() )
    

In [22]:
def get_next_words(text, match, sep=' '):
    words = iter(text.split(sep))
    for word in words:
        if word == match:
            yield next(words)
            
text = htmlmarkdown
words = get_next_words(text, 'docs')

described_columns_list = []

for w in words:
    
    described_columns_list.append (w)

In [24]:
git_path

'/Users/lewischarlesbaker/Documents/GitHub/droughty'

In [27]:
documents_path = 'docs'

In [30]:
print(documents_path)

None


In [23]:
described_columns_list

['web_event_pk',
 'web_event_session_fk',
 'web_event_compound_key',
 'web_event_row_no_per_session',
 'web_event_bundle_sequence_natural_key',
 'web_visitor_natural_key',
 'web_user_natural_key',
 'web_event_session_natural_key',
 'web_event_vehicle_deal_natural_key',
 'web_event_ecommerce_transaction_natural_key',
 'web_event_data_source',
 'web_event_session_number',
 'web_event_name',
 'web_event_page_type',
 'web_event_page_location',
 'web_event_page_title',
 'web_event_page_referrer',
 'web_event_page_referrer_ignore',
 'web_event_entrance',
 'web_event_link_text',
 'web_event_link_domain',
 'web_link_url',
 'web_event_channel',
 'web_event_source',
 'web_event_medium',
 'web_event_campaign',
 'web_event_content',
 'web_event_term',
 'web_event_visitor_first_campaign',
 'web_event_visitor_first_medium',
 'web_event_visitor_first_source',
 'web_event_device_category',
 'web_event_device_mobile_brand_name',
 'web_event_device_mobile_model_name',
 'web_event_device_mobile_marketing